In [24]:
import cv2

test = cv2.imread("~/D-NeRF/data/EXR_RGBD/depth" + f"0.exr",  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH) 
print(test)
cv2.imshow('exr', test)

None


error: OpenCV(4.5.5) /io/opencv/modules/highgui/src/window.cpp:1000: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
